In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
from scipy.ndimage import convolve
import matplotlib.animation as animation
import warnings
import json
import scipy
import os
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import StratifiedKFold

import discrete_CRISPR_methods as ds
import discrete_CRISPR_sim_methods as ds2

In [2]:
def calculate_variance(single_frame, params, sim_params):
    x_range = sim_params["xdomain"] #Initialize the spaces
    dx = sim_params["dx"]

    x_linspace = np.arange(-x_range, x_range, dx)
    coordmap = np.meshgrid(x_linspace, x_linspace)
    radius = coordmap[0]**2 + coordmap[1]**2

    rqrd = 0
    for n, r2 in zip(np.ravel(single_frame), np.ravel(radius)):
        rqrd += n*r2
    
    return rqrd/np.sum(single_frame)

In [88]:
def get_Gaussian_Fit(frames_n, params, sim_params):
    frame_means = []
    frame_cov = []
    x_range = sim_params["xdomain"] #Initialize the spaces
    dx = sim_params["dx"]

    x_linspace = np.arange(-x_range, x_range, dx)
    coordmap = np.meshgrid(x_linspace, x_linspace)

    for single_frame in frames_n:
        point_data = []
        for nonzero_ind in np.argwhere(single_frame):
            n = single_frame[nonzero_ind[0], nonzero_ind[1]]
            x_ind = coordmap[1][nonzero_ind[0], nonzero_ind[1]]
            y_ind = coordmap[0][nonzero_ind[0], nonzero_ind[1]]
            for _ in range(n):
                point_data.append([x_ind, y_ind])

        point_data = np.array(point_data).squeeze()

        gaussian_estimator = GaussianMixture(covariance_type="full", max_iter=20)
        gaussian_estimator.fit(point_data)

        covariances = gaussian_estimator.covariances_.squeeze()
        means = gaussian_estimator.means_.squeeze()

        frame_means.append(means)
        frame_cov.append(covariances)

    frame_means = np.array(frame_means).squeeze()
    return frame_means, frame_cov

In [89]:
def make_ellipse(means, covariances, color = "navy"):
    v, w = np.linalg.eigh(covariances)
    u = w[0] / np.linalg.norm(w[0])
    angle = np.arctan2(u[1], u[0])
    angle = 180 * angle / np.pi  # convert to degrees
    v = 2.0 * np.sqrt(2.0) * np.sqrt(v)
    ell = mpl.patches.Ellipse(
        means, v[0], v[1], 180 + angle, color=color
    )
    return ell

In [3]:
foldername = '/media/homes/thisiszhi/code/data/ParamsSweep/'
subfolders = [ f.path for f in os.scandir(foldername) if f.is_dir() ]

In [98]:
for current_folder in subfolders:
    print(current_folder)
    os.chdir(current_folder)
    params = json.load(open("_params.json"))
    sim_params = json.load(open("_sim_params.json"))

    frames_f = np.load("frames_f.npy")
    frames_n = np.load("frames_n.npy")

    frames_nvar = []
    for frame in frames_n:
        frames_nvar.append(calculate_variance(frame, params, sim_params))

    frames_nvar = np.array(frames_nvar).squeeze()
    avg_var = np.mean(frames_nvar)

    plt.figure()
    plt.title("$<r^2(t)>$ of N")
    plt.ylabel("dist")
    plt.xlabel("frame number")
    plt.plot(frames_nvar)
    plt.hlines(avg_var, 0, len(frames_nvar), colors="r")
    plt.savefig("variance_plot.png")
    plt.close()

/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#7
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#3
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#5
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#1
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#0
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#9
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#8
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#2
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#6
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#4


In [59]:
for current_folder in subfolders:
    print(current_folder)
    os.chdir(current_folder)
    params = json.load(open("_params.json"))
    sim_params = json.load(open("_sim_params.json"))

    frames_f = np.load("frames_f.npy")
    frames_n = np.load("frames_n.npy")

    frame = ma.array(frames_f[0], mask = (frames_n[0] == 0)).compressed()
    height, bins = np.histogram(frame, 25, range = (-1.1, 1.1))
    cummulative = np.zeros(height.shape)

    for single_f, single_n in zip(frames_f, frames_n):
        frame = ma.array(single_f, mask = (single_n == 0)).compressed()
        height, _ = np.histogram(frame, 25, range = (-1.1, 1.1))
        cummulative += height

    norm_height = cummulative/np.sum(cummulative)
    plt.figure()
    plt.bar(bins[:-1], norm_height, width = 0.05)
    plt.title("Fitness Distribution (entire frame)")
    plt.savefig("fitness_plot.png")
    plt.close()

/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#7
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#3
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#5
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#1
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#0
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#9
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#8
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#2
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#6
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#4


In [57]:
for current_folder in subfolders:
    print(current_folder)
    os.chdir(current_folder)
    params = json.load(open("_params.json"))
    sim_params = json.load(open("_sim_params.json"))

    frames_n = np.load("frames_n.npy")

    N = []

    for single_n in frames_n:
        n = np.sum(single_n)
        N.append(n)

    plt.figure()
    plt.plot(N)
    plt.title("Number of active phage over time")
    plt.ylabel("Population")
    plt.xlabel("Frame number")
    plt.savefig("Inflected_Population_plot.png")
    plt.close()

/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#7
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#3
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#5
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#1
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#0
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#9
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#8
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#2
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#6
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#4


In [99]:
for current_folder in subfolders:
    print(current_folder)
    os.chdir(current_folder)
    params = json.load(open("_params.json"))
    sim_params = json.load(open("_sim_params.json"))
    colors = ["navy", "turquoise", "darkorange"]

    frames_n = np.load("frames_n.npy")
    frames_means, frames_cov = get_Gaussian_Fit(frames_n, params, sim_params)

    fig, ax = plt.subplots()
    ax.plot(frames_means[:,0], frames_means[:,1])
    plt.title("Trajectory of Gaussian Fit")

    ax.scatter(frames_means[0, 0], frames_means[0, 1], color = 'red')
    ell1 = make_ellipse(frames_means[0], frames_cov[0], 'red')
    ell1.set_clip_box(ax.bbox)
    ell1.set_alpha(0.15)
    ax.add_patch(ell1)
    ax.set_aspect("equal", "datalim")

    ax.scatter(frames_means[-1, 0], frames_means[-1, 1], color = 'black')
    ell2 = make_ellipse(frames_means[-1], frames_cov[-1], 'black')
    ell2.set_clip_box(ax.bbox)
    ell2.set_alpha(0.15)
    ax.add_patch(ell2)
    ax.set_aspect("equal", "datalim")
    plt.savefig("Gaussian_Fit_Plot.png")
    plt.close()

/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#7
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#3
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#5
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#1
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#0
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#9
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#8
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#2
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#6
/media/homes/thisiszhi/code/data/ParamsSweep/Simulation#4
